In [23]:
import pandas as pd
import os 
from scipy.stats import spearmanr, pearsonr
from statsmodels.stats.multitest import multipletests
from scipy import stats

path = os.path.join(os.getcwd(), "data")
output_path = os.path.join(os.getcwd(),"analysis_output")
tax_data = pd.read_csv(path+"/PC_normalized_PC92_HC384_130 genus_ML.csv")
path_data = pd.read_csv(path+"/PC Pathway.csv")
ortho_data = pd.read_csv(path+'/PC Orthology.csv')
mapping_df = pd.read_excel(path+ "/pathway_Orthology_map.xlsx")
ortho_label = pd.read_excel(path+'/orthology_label_list.xlsx')
tax_data
print(path_data.shape)
print(tax_data.shape)
print(ortho_data.shape)
print(mapping_df.shape)

(476, 449)
(476, 133)
(476, 14875)
(1552, 2)


In [6]:
tax_data

,study_no,group,group_1,Abiotrophia,Acetatifactor,Acholeplasmataceae_uc,Actinobacillus,Actinobaculum,Actinomyces,Aggregatibacter,...,Xanthomonas,Cloacibacterium,Legionella,Variovorax,Bergeriella,Ethanoligenens,Lentimicrobiaceae_uc,Moraxella,Mycoplasma_g13,Streptobacillus
0,H1700097,Cancer,1,0.00,0.0,0.0,0.00,0.00,3.70,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,H1700105,Cancer,1,0.00,0.0,0.0,0.00,0.00,0.70,0.90,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,H1700084,Cancer,1,0.00,0.0,0.0,0.00,0.00,1.30,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,H1700109,Cancer,1,0.00,0.0,0.0,0.00,0.10,0.80,0.00,...,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,H1700098,Cancer,1,0.00,0.0,0.0,0.00,0.00,2.50,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,H1900931,Control,0,0.00,0.0,0.0,0.03,0.06,0.94,0.68,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
472,H1900932,Control,0,0.00,0.0,0.0,0.00,0.04,0.94,0.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
473,H1900933,Control,0,0.00,0.0,0.0,0.00,0.02,0.45,0.93,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
474,H1900936,Control,0,0.37,0.0,0.0,0.00,0.00,1.82,0.03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


데이터 구조 확인

In [7]:
# Set 'study_no' as the index for each DataFrame
tax_data.set_index('study_no', inplace=True)
path_data.set_index('study_no', inplace=True)
ortho_data.set_index('study_no', inplace=True)

# Sort each DataFrame by the index in ascending order
tax_data.sort_index(inplace=True)
path_data.sort_index(inplace=True)
ortho_data.sort_index(inplace=True)


# 기존 Taxonomy Analysis 결과에 따른 genus 선정

### Up-regulated 된 genus와 down-regulated 된 genus 데이터 추출

In [ ]:

use_genus = ['group_1','Desulfovibrio', 'Fretibacterium', 'Lactobacillus', 'Leuconostoc', 
                      'Olsenella', 'Parvimonas', 'Ralstonia','Pseudomonas', 'Simonsiella']
use_genus_data = tax_data[use_genus]
pc_tax_data = use_genus_data[use_genus_data['group_1'] == 1].drop('group_1', axis=1)
control_tax_data = use_genus_data[use_genus_data['group_1'] == 0].drop('group_1', axis=1)

,Desulfovibrio,Fretibacterium,Lactobacillus,Leuconostoc,Olsenella,Parvimonas,Ralstonia,Pseudomonas,Simonsiella
study_no,,,,,,,,,
H1700001,0.00,0.48,0.00,0.03,1.71,9.06,0.00,0.0,0.0
H1700002,0.00,0.00,0.05,0.00,0.02,0.19,0.00,0.0,0.0
H1700003,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.0,0.0
H1700004,0.04,0.24,0.23,0.00,0.35,6.24,0.00,0.0,0.0
H1700005,0.00,0.15,0.24,0.00,0.03,0.13,0.02,0.0,0.0
...,...,...,...,...,...,...,...,...,...
H1700106,0.00,0.60,0.00,0.00,0.00,0.40,0.00,0.0,0.0
H1700107,0.00,2.10,0.00,0.00,0.00,0.50,0.00,0.0,0.0
H1700108,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.0,0.0


### Pathway 통계 분석을 통해 선정된 up-regulated된 pathway와 down-regulated된 pathway 선정

In [25]:
path_data = pd.read_excel(output_path+'/statistical_analysis_results_with_labels_pathway.xlsx',index_col = 0)
ortho_data = pd.read_excel(output_path+'/statistical_analysis_results_with_labels_Orthology.xlsx',index_col = 0)
data = path_data.dropna(subset=['log2FC_value', 'Wilcoxon_p_value(-log10)'])
data2 = ortho_data.dropna(subset = ['log2FC_value', 'Wilcoxon_p_value(-log10)'])

In [26]:
fold_change_threshold = 1.0  # threshold for log2 fold change
significance_threshold = 1.3  # threshold for -log10(p-value) (~p < 0.05)
logistic_threshold = 0.05  # threshold for logistic Regression with false discovery rate adjusted p-value


In [ ]:

# Based on statistical analysis from pathway bring out upregulated pathway in Cancer group and downregulated pathway in Cancer group
# Correcting the formatting of down_path and up_path lists
up_path = data.loc[(data['log2FC_value'] > fold_change_threshold) & 
         (data['Wilcoxon_p_value(-log10)'] > significance_threshold) &
         (data['LogReg_p_fdr'] < logistic_threshold), 'color'] = 'red'
down_path = data.loc[(data['log2FC_value'] < -fold_change_threshold) & 
         (data['Wilcoxon_p_value(-log10)'] > significance_threshold) &
         (data['LogReg_p_fdr'] < logistic_threshold),
up_path = [
    'ko04024', 'ko04977', 'ko00601', 'ko04530', 'ko04151',
    'ko04141', 'ko01053', 'ko00220', 'ko04120'
]
use_path ='[ group_2',''
# Filtering the path_data DataFrame for the upregulated and downregulated pathways
up_path_data = path_data[up_path]
down_path_data = path_data[down_path]
print(up_path_data.shape, down_path_data.shape)


(476, 9) (476, 15)


In [ ]:
selected_path = path_data[up_path+down_path]
selected_path

,ko04024,ko04977,ko00601,ko04530,ko04151,ko04141,ko01053,ko00220,ko04120,ko00472,...,ko03320,ko00333,ko00471,ko04750,ko00830,ko00944,ko00404,ko04330,ko00565,ko04975
study_no,,,,,,,,,,,,,,,,,,,,,
H1700001,0.000321,0.000299,0.000184,0.000992,0.001175,0.000683,0.000343,0.003190,0.000468,0.000154,...,0.001280,0.001200,0.001452,0.000199,0.000728,0.000032,0.000008,0.000038,0.000347,0.000038
H1700002,0.000228,0.000114,0.000115,0.000411,0.000665,0.000561,0.000465,0.003634,0.000304,0.000111,...,0.000905,0.001383,0.001380,0.000179,0.000539,0.000013,0.000030,0.000064,0.000193,0.000009
H1700003,0.000772,0.000160,0.000055,0.000402,0.000877,0.000403,0.000836,0.004040,0.000826,0.000097,...,0.000846,0.001387,0.001205,0.000288,0.000722,0.000016,0.000105,0.000010,0.000468,0.000007
H1700004,0.000159,0.000171,0.000093,0.000362,0.000662,0.000578,0.000480,0.003526,0.000244,0.000136,...,0.001106,0.001286,0.001492,0.000134,0.000530,0.000019,0.000002,0.000033,0.000156,0.000019
H1700005,0.000202,0.000112,0.000138,0.000334,0.000645,0.000787,0.000502,0.003485,0.000328,0.000090,...,0.001230,0.001375,0.001387,0.000153,0.000357,0.000019,0.000011,0.000047,0.000135,0.000013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H1900931,0.000019,0.000015,0.000022,0.000126,0.000190,0.000258,0.000243,0.001628,0.000261,0.003436,...,0.003813,0.004632,0.004155,0.000743,0.001940,0.001301,0.000717,0.002465,0.003482,0.000257
H1900932,0.000015,0.000012,0.000020,0.000174,0.000256,0.000229,0.000279,0.001748,0.000292,0.003323,...,0.003710,0.004256,0.003987,0.000836,0.001826,0.001867,0.000796,0.002764,0.003162,0.000270
H1900933,0.000017,0.000006,0.000023,0.000118,0.000222,0.000256,0.000246,0.001847,0.000279,0.003322,...,0.003726,0.004564,0.004190,0.000701,0.001915,0.001220,0.000740,0.002572,0.003535,0.000285


In [ ]:
from scipy import stats
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import multipletests

# Step 1: Filter taxonomy and pathway data based on selected columns
def filter_data(data, selected_columns):
    return data[selected_columns]



# Step 2: Check normality of data using Shapiro-Wilk test
def check_normality(series, alpha=0.05):
    return stats.shapiro(series).pvalue > alpha

# Step 3: Compute correlation based on normality
def apply_correlation(x, y, alpha=0.05):
    # Ensure x and y are 1-dimensional
    if x.ndim > 1:
        x = x.squeeze()  # Convert to 1D if possible
    if y.ndim > 1:
        y = y.squeeze()  # Convert to 1D if possible
    
    # Verify that x and y are now 1D
    if x.ndim != 1 or y.ndim != 1:
        raise ValueError("Both x and y must be 1-dimensional for correlation computation.")
    
    # Apply correlation based on normality
    if check_normality(x, alpha) and check_normality(y, alpha):
        corr, p_value = stats.pearsonr(x, y)
        method = "Pearson"
    else:
        corr, p_value = stats.spearmanr(x, y)
        method = "Spearman"

    # Ensure that corr and p_value are scalar values, not matrices
    if isinstance(corr, np.ndarray) or isinstance(p_value, np.ndarray):
        raise ValueError("Correlation and p-value should be scalars, not matrices.")

    return corr, p_value, method

# Step 4: Adjust p-values using Benjamini-Hochberg FDR
def adjust_p_values(p_values):
    _, p_adj_all, _, _ = multipletests(p_values, method='fdr_bh')
    return p_adj_all

# Main function to calculate correlations and filter significant results
def calculate_correlation(taxonomy_data, pathway_data, selected_genus, up_path, down_path, label="Pathway", alpha=0.05):
    taxonomy_selected = filter_data(taxonomy_data, selected_genus)
    up_path_data = filter_data(pathway_data, up_path)
    down_path_data = filter_data(pathway_data, down_path)

    results = []

    # Step 5: Run the correlation computation for each pair
    for tax_col in taxonomy_selected.columns:
        for path_col in up_path_data.columns:
            corr, p_value, method = apply_correlation(taxonomy_selected[tax_col], up_path_data[path_col], alpha)
            results.append((tax_col, path_col, corr, p_value, method, 'Upregulated'))
        
        for path_col in down_path_data.columns:
            corr, p_value, method = apply_correlation(taxonomy_selected[tax_col], down_path_data[path_col], alpha)
            results.append((tax_col, path_col, corr, p_value, method, 'Downregulated'))

    # Step 6: Adjust p-values and filter significant correlations
    p_values = [result[3] for result in results]
    p_adj = adjust_p_values(p_values)

    # Add adjusted p-values to results without modifying values with the label
    final_results = []
    for i, (tax_col, path_col, corr, p_value, method, reg_type) in enumerate(results):
        final_results.append((tax_col, path_col, corr, p_value, p_adj[i], method, reg_type))

    # Convert to DataFrame with dynamic column names
    column_names = ["Taxonomy", label, f"{label}_Correlation", 
                    f"{label}_p_value", f"{label}_p_value_adjusted", 
                    f"{label}_Correlation_Method", f"{label}_Regulation_Type"]
    
    correlation_df = pd.DataFrame(final_results, columns=column_names)

    # Filter for significant correlations
    significant_corr_df = correlation_df[(correlation_df[f'{label}_p_value_adjusted'] < alpha) & 
                                         (abs(correlation_df[f'{label}_Correlation']) > 0.2)]

    return significant_corr_df


In [ ]:
up_regulated_genus = calculate_correlation(tax_data, path_data,up_genus, up_path, down_path)
down_regulated_genus = calculate_correlation(tax_data, path_data, down_genus, up_path, down_path)
# Display the significant correlations
print("Significant Correlations:")
print(up_regulated_genus)
print(down_regulated_genus)


Significant Correlations:
          Taxonomy  Pathway  Pathway_Correlation  Pathway_p_value  \
0    Desulfovibrio  ko04024             0.238176     1.451721e-07   
3    Desulfovibrio  ko04530             0.237698     1.539914e-07   
4    Desulfovibrio  ko04151             0.228665     4.589294e-07   
5    Desulfovibrio  ko04141             0.213168     2.697765e-06   
6    Desulfovibrio  ko01053             0.200167     1.081236e-05   
..             ...      ...                  ...              ...   
151      Ralstonia  ko00220             0.230607     3.642803e-07   
154      Ralstonia  ko05231            -0.208087     4.689993e-06   
155      Ralstonia  ko00430            -0.214795     2.253267e-06   
160      Ralstonia  ko00471            -0.212862     2.790216e-06   
162      Ralstonia  ko00830            -0.226409     5.987259e-07   

     Pathway_p_value_adjusted Pathway_Correlation_Method  \
0                3.933695e-07                   Spearman   
3                4.106437

In [ ]:

pathway_label = pd.read_csv(path+"pathway_label_list.csv")
pathway_label

,KEGG_no,pathway_kegg_no
0,ko00010,Glycolysis / Gluconeogenesis
1,ko00020,Citrate cycle (TCA cycle)
2,ko00030,Pentose phosphate pathway
3,ko00040,Pentose and glucuronate interconversions
4,ko00051,Fructose and mannose metabolism
...,...,...
441,ko05410,Hypertrophic cardiomyopathy (HCM)
442,ko05412,Arrhythmogenic right ventricular cardiomyopath...
443,ko05414,Dilated cardiomyopathy (DCM)
444,ko05416,Viral myocarditis


In [ ]:

# Assuming you have two DataFrames:
# 1. `correlation_df`: The correlation results DataFrame (your provided correlation results)
# 2. `kegg_df`: The KEGG pathway dataset with `KEGG_no` and `pathway_kegg_no`

# Example DataFrames (replace with your actual data)
# correlation_df = pd.read_csv('correlation_results.csv')  # Load your correlation results if from CSV

# Perform the merge operation to add pathway labels to your correlation results
up_merged_df = pd.merge(up_regulated_genus, pathway_label, left_on='Pathway', right_on='KEGG_no', how='left')

# Drop the now redundant 'KEGG_no' column
up_merged_df.drop(columns=['KEGG_no'], inplace=True)

# Optionally rearrange columns for readability
up_merged_df = up_merged_df[["Taxonomy", "Pathway", "pathway_kegg_no","Pathway_Correlation", "Pathway_p_value", "Pathway_p_value_adjusted", "Pathway_Correlation_Method", "Pathway_Regulation_Type"]]

down_merged_df = pd.merge(down_regulated_genus, pathway_label, left_on='Pathway', right_on='KEGG_no', how='left')

# Drop the now redundant 'KEGG_no' column
down_merged_df.drop(columns=['KEGG_no'], inplace=True)
down_merged_df = down_merged_df[["Taxonomy", "Pathway", "pathway_kegg_no","Pathway_Correlation", "Pathway_p_value", "Pathway_p_value_adjusted", "Pathway_Correlation_Method", "Pathway_Regulation_Type"]]


In [ ]:
up_path_name = up_merged_df[up_merged_df["Pathway"].isin(up_path)]
up_path_name.pathway_kegg_no.unique()
down_path_name = up_merged_df[up_merged_df["Pathway"].isin(down_path)]
down_path_name.pathway_kegg_no.unique()


array(['D-Arginine and D-ornithine metabolism',
       'Choline metabolism in cancer',
       'Taurine and hypotaurine metabolism', 'Prodigiosin biosynthesis',
       'D-Glutamine and D-glutamate metabolism', 'Retinol metabolism',
       'Ether lipid metabolism', 'Steroid degradation',
       'mTOR signaling pathway',
       'Inflammatory mediator regulation of TRP channels',
       'Flavone and flavonol biosynthesis', 'Notch signaling pathway',
       'PPAR signaling pathway', 'Staurosporine biosynthesis',
       'Fat digestion and absorption'], dtype=object)

### 기존 선정되었던 orthology 선정

In [ ]:
selected_ortho_data = pd.read_excel(output_path+'/orthology_analysis_results.xlsx',index_col = 0)
selected_ortho_list = selected_ortho_data['Orthology_kegg_no']

### 그 중 모든 값이 0인 orthology 제거

In [ ]:
sod = ortho_data[selected_ortho_list]
sod = sod.loc[:, sod.sum() != 0]
label = ortho_data.iloc[:,:3]
# Merge based on the "index" column
otho_df = pd.merge(label, sod, how='left', left_index=True, right_index=True)
otho_df

,group_1,group_2,K00001_x,K00001_y,K00029,K00059,K00061,K00121,K00145,K00157,...,K21791,K21792,K22476,K22477,K22478,K08267,K20403,K18435,K09554,K22387
study_no,,,,,,,,,,,,,,,,,,,,,
H1700001,Cancer,1,0.000152,0.000152,0.000256,0.001308,0.0,0.000029,0.000098,1.880000e-06,...,5.960000e-06,0.0,1.090000e-06,0.000086,0.0,0.0,0.0,0.0,0.0,0.0
H1700002,Cancer,1,0.000255,0.000255,0.000385,0.001398,0.0,0.000050,0.000205,0.000000e+00,...,1.800000e-07,0.0,0.000000e+00,0.000010,0.0,0.0,0.0,0.0,0.0,0.0
H1700003,Cancer,1,0.000121,0.000121,0.000641,0.001466,0.0,0.000226,0.000250,1.100000e-07,...,2.000000e-08,0.0,3.600000e-07,0.000009,0.0,0.0,0.0,0.0,0.0,0.0
H1700004,Cancer,1,0.000220,0.000220,0.000242,0.001331,0.0,0.000010,0.000139,5.300000e-07,...,3.140000e-06,0.0,1.100000e-06,0.000036,0.0,0.0,0.0,0.0,0.0,0.0
H1700005,Cancer,1,0.000143,0.000143,0.000473,0.001274,0.0,0.000017,0.000140,2.300000e-07,...,2.500000e-07,0.0,3.300000e-07,0.000009,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H1900931,Control,0,0.000076,0.000076,0.000641,0.001260,0.0,0.000109,0.000148,6.000000e-08,...,3.000000e-08,0.0,7.200000e-07,0.000014,0.0,0.0,0.0,0.0,0.0,0.0
H1900932,Control,0,0.000051,0.000051,0.000733,0.001354,0.0,0.000194,0.000163,0.000000e+00,...,2.100000e-07,0.0,1.200000e-06,0.000013,0.0,0.0,0.0,0.0,0.0,0.0
H1900933,Control,0,0.000126,0.000126,0.000593,0.001266,0.0,0.000104,0.000192,2.200000e-07,...,1.900000e-07,0.0,1.110000e-06,0.000025,0.0,0.0,0.0,0.0,0.0,0.0


### 위에서 확인된 789개의 orthology와 genus간의 Correlation analysis 진행

In [ ]:
selected_ortho_data.dropna().reset_index(drop=True,inplace=True)

In [ ]:
up_regulated_ortho = selected_ortho_data[(selected_ortho_data['FC_value'] >= 2) & (selected_ortho_data['Wilcoxon_p'] <= 0.05) & (selected_ortho_data['LogReg_p_fdr'] <= 0.05)]
down_regulated_ortho = selected_ortho_data[(selected_ortho_data['FC_value'] <= 0.5) & (selected_ortho_data['Wilcoxon_p'] <= 0.05) & (selected_ortho_data['LogReg_p_fdr'] <= 0.05)]

In [ ]:
up_regulated_ortho.to_excel(output_path + "/up_regulated_ortho.xlsx")
down_regulated_ortho.to_excel(output_path+ "/down_regulated_ortho.xlsx")

In [ ]:
down_ortho = down_regulated_ortho.Orthology_kegg_no.to_list()
up_ortho = up_regulated_ortho.Orthology_kegg_no.unique().tolist()

In [ ]:
down_regulated_ortho

,Pathway_ID,Orthology_kegg_no,Orthology_Name,Control_mean,Cancer_mean,FC_value,Wilcoxon_p,log2FC_value,LogReg_p_univ,LogReg_p_fdr
149,ko00430,K03119,taurine dioxygenase,0.000002,6.757609e-07,0.283400,3.287933e-02,-1.819089,4.375793e-20,6.127878e-18
453,ko00601,K07966,"beta-1,4-galactosyltransferase 1",0.000005,1.778478e-06,0.369554,5.579062e-16,-1.436143,1.135174e-15,3.265155e-14
522,ko04141,K09523,DnaJ homolog subfamily C member 3,0.000031,1.475500e-05,0.475648,9.928891e-13,-1.072034,3.316219e-06,2.053419e-05
581,ko04120,K10610,DNA damage-binding protein 1,0.000005,1.563478e-06,0.312862,2.532964e-09,-1.676402,1.087053e-07,8.226478e-07


In [ ]:
up_tax_ortho_correlation = calculate_correlation(tax_data, ortho_data,up_genus, up_ortho, down_ortho,label='Orthology')
down_tax_ortho_correlation = calculate_correlation(tax_data, ortho_data, down_genus, up_ortho, down_ortho, label="Orthology")

In [ ]:
up_tax_ortho_correlation.to_excel(output_path+"/Correlation_analysis_up_genus_orthology.xlsx",index=False)
down_tax_ortho_correlation.to_excel(output_path+"/Correlation_analysis_down_genus_orthology.xlsx",index=False)

In [ ]:
ortho_label

,Ortholog,Definition
0,K12369,dipeptide transport system permease protein
1,K02389,flagellar basal-body rod modification protein ...
2,K00839,(S)-ureidoglycine---glyoxylate transaminase
3,K15345,secreted effector protein SseC
4,K06977,uncharacterized protein
...,...,...
12494,K15744,zeta-carotene isomerase
12495,K12705,8-demethylnovobiocic acid C8-methyltransferase
12496,K11534,"DeoR family transcriptional regulator, deoxyri..."
12497,K17262,tubulin-specific chaperone B


In [ ]:
# Merge and drop 'Ortholog' in one step
up_tax_ortho_correlation = (
    pd.merge(up_tax_ortho_correlation, ortho_label, how='left', left_on='Orthology', right_on='Ortholog')
    .drop('Ortholog', axis=1)
)
down_tax_ortho_correlation = (
    pd.merge(down_tax_ortho_correlation, ortho_label, how = 'left', left_on = 'Orthology', right_on = 'Ortholog').drop("Ortholog",axis=1)
)

# Rearrange the columns
cols = up_tax_ortho_correlation.columns
up_tax_ortho_correlation = up_tax_ortho_correlation[[cols[0], cols[1], cols[-1]] + list(cols[2:-1])]
down_tax_ortho_correlation = down_tax_ortho_correlation[[cols[0], cols[1], cols[-1]] + list(cols[2:-1])]


In [ ]:
mapping_df = pd.read_excel(path+'/pathway_Orthology_map.xlsx',sheet_name='Sheet2')
# Remove all kinds of whitespace (including \xa0) from 'Orthology_kegg_no' column in mapping_df
mapping_df['Orthology_kegg_no'] = mapping_df['Orthology_kegg_no'].str.replace(r'\s+', '', regex=True)


In [ ]:
'K00485' in mapping_df['Orthology_kegg_no'].values
    

True

In [ ]:
tmp1 = pd.merge(
    up_tax_ortho_correlation,
    mapping_df[['Pathway_ID', 'Orthology_kegg_no']],
    how='left',
    left_on="Orthology",
    right_on="Orthology_kegg_no"
)
tmp2 = pd.merge(
    down_tax_ortho_correlation,
    mapping_df[['Pathway_ID', 'Orthology_kegg_no']],
    how='left',
    left_on="Orthology",
    right_on="Orthology_kegg_no"
)

In [ ]:
# Correct way to merge on multiple columns
up_results = pd.merge(
    up_merged_df,
    tmp1,
    how='right',
    left_on=['Taxonomy', 'Pathway'],
    right_on=['Taxonomy', 'Pathway_ID']
)
down_results = pd.merge(
    down_merged_df,
    tmp2,
    how='right',
    left_on=['Taxonomy', 'Pathway'],
    right_on=['Taxonomy', 'Pathway_ID']
)

In [ ]:
up_results.to_excel(output_path+"/up_results.xlsx")
down_results.to_excel(output_path+"/down_results.xlsx")

In [ ]:
up_results

,Taxonomy,Pathway,pathway_kegg_no,Pathway_Correlation,Pathway_p_value,Pathway_p_value_adjusted,Pathway_Correlation_Method,Pathway_Regulation_Type,Orthology,Definition,Orthology_Correlation,Orthology_p_value,Orthology_p_value_adjusted,Orthology_Correlation_Method,Orthology_Regulation_Type,Pathway_ID,Orthology_kegg_no
0,Desulfovibrio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K00485,dimethylaniline monooxygenase (N-oxide forming),0.244840,6.291979e-08,2.202193e-07,Spearman,Upregulated,ko00430,K00485
1,Desulfovibrio,ko00220,Arginine biosynthesis,0.206622,5.486889e-06,1.084467e-05,Spearman,Upregulated,K01476,arginase,0.221214,1.092854e-06,3.059992e-06,Spearman,Upregulated,ko00220,K01476
2,Desulfovibrio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K03371,"alpha-N-acetyl-neuraminate alpha-2,8-sialyltra...",0.311932,3.351986e-12,1.932321e-11,Spearman,Upregulated,ko00601,K03371
3,Desulfovibrio,ko04024,cAMP signaling pathway,0.238176,1.451721e-07,3.933695e-07,Spearman,Upregulated,K05208,"glutamate receptor ionotropic, NMDA 1",0.284686,2.513500e-10,1.172967e-09,Spearman,Upregulated,ko04024,K05208
4,Fretibacterium,ko00430,Taurine and hypotaurine metabolism,-0.255149,1.644366e-08,6.005512e-08,Spearman,Downregulated,K00485,dimethylaniline monooxygenase (N-oxide forming),0.425210,2.536291e-22,2.761739e-21,Spearman,Upregulated,ko00430,K00485
5,Fretibacterium,ko00220,Arginine biosynthesis,0.353130,1.999101e-15,3.053172e-14,Spearman,Upregulated,K01476,arginase,0.483422,3.016141e-29,4.926364e-28,Spearman,Upregulated,ko00220,K01476
6,Fretibacterium,ko00601,Glycosphingolipid biosynthesis - lacto and neo...,0.239896,1.172644e-07,3.339054e-07,Spearman,Upregulated,K03371,"alpha-N-acetyl-neuraminate alpha-2,8-sialyltra...",0.656058,6.511411e-60,3.190591e-58,Spearman,Upregulated,ko00601,K03371
7,Fretibacterium,ko00430,Taurine and hypotaurine metabolism,-0.255149,1.644366e-08,6.005512e-08,Spearman,Downregulated,K03852,sulfoacetaldehyde acetyltransferase,0.294357,5.718457e-11,2.802044e-10,Spearman,Upregulated,ko00430,K03852
8,Fretibacterium,ko04024,cAMP signaling pathway,0.403474,4.623776e-20,2.102678e-18,Spearman,Upregulated,K05208,"glutamate receptor ionotropic, NMDA 1",0.542645,8.438361e-38,2.067398e-36,Spearman,Upregulated,ko04024,K05208
9,Fretibacterium,ko04151,PI3K-Akt signaling pathway,0.335350,5.649018e-14,7.300269e-13,Spearman,Upregulated,K07207,tuberous sclerosis 2,0.248362,4.004823e-08,1.453603e-07,Spearman,Upregulated,ko04151,K07207


In [ ]:
down_results

,Taxonomy,Pathway,pathway_kegg_no,Pathway_Correlation,Pathway_p_value,Pathway_p_value_adjusted,Pathway_Correlation_Method,Pathway_Regulation_Type,Orthology,Definition,Orthology_Correlation,Orthology_p_value,Orthology_p_value_adjusted,Orthology_Correlation_Method,Orthology_Regulation_Type,Pathway_ID,Orthology_kegg_no
0,Pseudomonas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K07298,serine/threonine-protein kinase 11,0.251945,2.511328e-08,1.757930e-07,Spearman,Upregulated,ko04530,K07298
1,Pseudomonas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K07298,serine/threonine-protein kinase 11,0.251945,2.511328e-08,1.757930e-07,Spearman,Upregulated,ko04151,K07298
2,Pseudomonas,ko04150,mTOR signaling pathway,0.276996,7.838723e-10,6.270978e-09,Spearman,Downregulated,K07298,serine/threonine-protein kinase 11,0.251945,2.511328e-08,1.757930e-07,Spearman,Upregulated,ko04150,K07298
3,Pseudomonas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K03119,taurine dioxygenase,0.687885,5.730474e-68,1.604533e-66,Spearman,Downregulated,ko00430,K03119
4,Simonsiella,ko00404,Staurosporine biosynthesis,0.235679,1.973172e-07,1.052358e-06,Spearman,Downregulated,K16438,5-epimerase,0.207119,5.203497e-06,2.913958e-05,Spearman,Upregulated,ko00404,K16438
5,Simonsiella,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K07966,"beta-1,4-galactosyltransferase 1",0.312153,3.230549e-12,3.015179e-11,Spearman,Downregulated,ko00601,K07966
6,Simonsiella,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K09523,DnaJ homolog subfamily C member 3,0.356042,1.133038e-15,1.586253e-14,Spearman,Downregulated,ko04141,K09523


### 기존 Pathway 상관없이

In [ ]:
path_df = pd.read_excel(output_path+'/statistical_analysis_results_with_labels.xlsx')   
path_df

,Unnamed: 0,KEGG_no,pathway_kegg_no,Control_mean,Cancer_mean,FC_value,Wilcoxon_p,log2FC_value,LogReg_p_univ,LogReg_p_fdr
0,0,ko04612,Antigen processing and presentation,NaN,NaN,NaN,NaN,NaN,6.059294e-13,NaN
1,1,ko04926,Relaxin signaling pathway,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ko00623,Toluene degradation,NaN,NaN,NaN,NaN,NaN,1.612643e-19,NaN
3,3,ko04320,Dorso-ventral axis formation,NaN,NaN,NaN,NaN,NaN,3.682350e-01,NaN
4,4,ko04730,Long-term depression,NaN,NaN,NaN,NaN,NaN,5.133567e-01,NaN
...,...,...,...,...,...,...,...,...,...,...
441,441,ko00680,Methane metabolism,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442,442,ko04970,Salivary secretion,NaN,NaN,NaN,NaN,NaN,NaN,NaN
443,443,ko04016,MAPK signaling pathway - plant,NaN,NaN,NaN,NaN,NaN,8.642694e-01,NaN
444,444,ko00515,Mannose type O-glycan biosynthesis,NaN,NaN,NaN,NaN,NaN,1.000000e+00,NaN
